# [모듈 4.1] 평가 및 조건 단계 개발 (SageMaker Model Building Pipeline 평가 및 조건 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 모델 평가 개요 
- 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 2. 모델 평가 코드 확인
- 3. 모델 평가 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 훈련 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) **[로컬 노트북 인스턴스]**에서 [다커 컨테이너 없이] 스크립트로 
        - (2) **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
        - (3) SageMaker Model Building Pipeline 에서 모델 평가 및 조건 스텝 개발 및 실행
    
---

# 0. 모델 평가 개요

모델 평가는 훈련된 모델이 성능이 얼마나 나오는지를 검증 데이터 셋으로 평가하는 단계 입니다. 크게 아래와 같은 단계를 가집니다.
- 모델 훈련이 된 모델 아티펙트를 가져옴.
- 모델 아티펙트를 모델 객체로 로딩
- 검증할 검증 데이터 세트를 준비
- 모델 객체를 통하여 검증 데이터의 추론
- 추론 후에 평가 지표 (에: roc-auc) 의 값을 얻습니다.

# 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
---

In [24]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
# role = 'arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20201219T152596'

%store -r 
# %store

### 모델 아피텍트 위치 확인

In [25]:
print("train_model_artifact: \n", train_model_artifact)

train_model_artifact: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/pipelines-hgtk0mchs6dm-FraudScratchTrain-K8lnzVaWt8/output/model.tar.gz


# 2. 모델 평가 코드 확인


---

평가 스크립트는 `xgboost`를 사용하고 다음을 실행합니다.

* 모델을 로드합니다. 
* 테스트 데이터를 읽습니다. 
* 테스트 데이터에 대한 예측을 실행합니다. 
* mse 등을 포함하는 분류보고서를 작성합니다. 




In [26]:
!pygmentize src/evaluation.py

import json
import pathlib
import pickle
import tarfile

import joblib
import numpy as np
import pandas as pd
import argparse
import os
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'xgboost'])

import xgboost

from sklearn.metrics import roc_auc_score

import logging
import logging.handlers

def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  

logger = _get_logger()



if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument('--base_dir', type=str, default= "/opt/m

# 3. 모델 평가 스텝 개발 및 실행
---



## (1) 로컬에서 스크립트 실행


### 로컬 환경 구성
- 도커 컨테이너의 환경을 가상으로 구성하기 위해 아래 폴더 생성
    - `opt/ml/processing`
    - `opt/ml/processing/evaluation`
    - `opt/ml/processing/model`
    

- 모델 아티펙트 다운로드 하여 로컬에 저장
- 테스트 데이터 세트 다운로드 하여 로컬 저장

In [27]:
import os
base_dir = 'opt/ml/processing'
os.makedirs(base_dir, exist_ok=True)

output_evaluation_dir = 'opt/ml/processing/evaluation'
os.makedirs(output_evaluation_dir, exist_ok=True)

# 훈련 아티펙트를 다운로드 하여 로컬 저장
base_model_dir = 'opt/ml/processing/model'
base_model_path = f"{base_model_dir}/model.tar.gz"
os.makedirs(base_model_dir, exist_ok=True)

print("Download model artifact: ")
! aws s3 cp  {train_model_artifact} {base_model_dir}

# 테스트 데이터 세트의 파일 경로 기술
base_test_path = f"{test_preproc_data_uri}"
print("Test Data Location: \n", base_test_path)


Download model artifact: 
download: s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/pipelines-hgtk0mchs6dm-FraudScratchTrain-K8lnzVaWt8/output/model.tar.gz to opt/ml/processing/model/model.tar.gz
Test Data Location: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase02/preporc/test.csv


### 로컬에서 실행 위한 필수 정보 확인

In [28]:
print("base_dir: \n", base_dir)
print("base_model_path: \n", base_model_path)
print("base_test_path: \n", base_test_path)
print("output_evaluation_dir: \n", output_evaluation_dir)

base_dir: 
 opt/ml/processing
base_model_path: 
 opt/ml/processing/model/model.tar.gz
base_test_path: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase02/preporc/test.csv
output_evaluation_dir: 
 opt/ml/processing/evaluation


### 로컬에서 스크립트 실행

In [29]:
%%sh -s "$base_dir" "$base_model_path" "$base_test_path" "$output_evaluation_dir"
python src/evaluation.py \
--base_dir $1 \
--model_path $2 \
--test_path $3 \
--output_evaluation_dir $4


#############################################
args.model_path: opt/ml/processing/model/model.tar.gz
args.test_path: s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase02/preporc/test.csv
args.output_evaluation_dir: opt/ml/processing/evaluation
****** All folder and files under opt/ml/processing ****** 
('opt/ml/processing', ['evaluation', 'model'], [])
('opt/ml/processing/evaluation', [], ['evaluation.json'])
('opt/ml/processing/model', [], ['model.tar.gz'])
************************************************* 
model is loaded
test df sample 
:    fraud  ...  police_report_available_Yes
0      0  ...                            1
1      0  ...                            1

[2 rows x 59 columns]
Payload: 
 [[2.39014325e+04 3.62014325e+04 5.60000000e+01 ... 1.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [2.90952957e+04 7.36952957e+04 3.60000000e+01 ... 1.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [1.12328318e+04 2.50328318e+04 2.40000000e+01 ... 0.00000000e

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## (2) 로컬 다커 에서 모델 평가


### ScriptProcessor 의 기본 도커 컨테이너 지정
ScriptProcessor 의 기본 도커 컨테이너로 Scikit-learn를 기본 이미지를 사용함. 
- 사용자가 정의한 도커 컨테이너도 사용할 수 있습니다.

### ScriptProcessor 정의 및 실행

In [30]:
image_uri = f'366743142698.dkr.ecr.{region}.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'
print("image_uri: \n", image_uri )

image_uri: 
 366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3


In [31]:
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput


processing_instance_type = 'local'

eval_script_processor = SKLearnProcessor(
                                     framework_version= "0.23-1",
                                     role=role,
                                     instance_type=processing_instance_type,
                                     instance_count=1,
                                     base_job_name="script-fraud-scratch-eval",
                                    )



eval_script_processor.run(
                        inputs=[
                            ProcessingInput(
#                                source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
                                source= train_model_artifact,  # model_artifcat_path,
                                destination="/opt/ml/processing/model"
                            ),
                            ProcessingInput(
#                                 source=step_process.properties.ProcessingOutputConfig.Outputs[
#                                     "test"
#                                 ].S3Output.S3Uri,
                                source = test_preproc_dir_artifact, # prep_test_output,
                                destination="/opt/ml/processing/test"
                            )
                        ],
                        outputs=[
                            #ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
                            ProcessingOutput(source="/opt/ml/processing/evaluation"),                            
                        ],
                        code="src/evaluation.py",
)


Job Name:  script-fraud-scratch-eval-2021-08-25-12-59-50-772
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/training_jobs/pipelines-hgtk0mchs6dm-FraudScratchTrain-K8lnzVaWt8/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-08-25-11-37-38-266/output/test', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/script-fraud-scratch-eval-2021-08-25-12-59-50-772/input/code/evaluation.py', 'L

## (3) 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [32]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

### 학습모델을 평가하기 위한 모델 평가단계 정의 



In [33]:
from sagemaker.processing import ScriptProcessor


script_eval = SKLearnProcessor(
                             framework_version= "0.23-1",
                             role=role,
                             instance_type=processing_instance_type,
                             instance_count=1,
                             base_job_name="script-fraud-scratch-eval",
                                    )


### Property 파일 정의

- step_eval 이 실행이 되면 `evaluation.json` 이 S3에 저장이 됩니다.
    - evaluation.json 은 아래의 PropertyFile 로서 정의 됩니다.
    - step_eval 에서 `property_files=[<property_file_instance>]` 를 추가 합니다.

```
<property_file_instance> = PropertyFile(
    name="<property_file_name>",
    output_name="<processingoutput_output_name>",
    path="<path_to_json_file>"
)
```


- 조건 단계에서 사용하는 ConditionLessThanOrEqualTo 에서 evaluation.json 을 로딩하여 내용을 확인

```
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=<property_file_instance>,
        json_path="test_metrics.roc.value",
    ),
    right=6.0
)
```

#### 참고
- 참고 자료: [Property Files and JsonGet](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-propertyfile.html)

In [34]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)


step_eval = ProcessingStep(
    name="FraudEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source= train_model_artifact,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source= test_preproc_dir_artifact,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="src/evaluation.py",
    property_files=[evaluation_report],
)

### 조건  단계 정의
- step_eval 의 결과가 조건 단계로 연결되기에 아래 추가하여 진행 합니다.
- 조건 단계의 상세 사항은 여기를 보세요. --> [조건 단계](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition)

In [35]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value",
    ),
    right=6.0
)

step_cond = ConditionStep(
    name="FraudMeticCond",
    conditions=[cond_lte],
#    if_steps=[step_register, step_create_model, step_transform],
    if_steps=[],    
    else_steps=[], 
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [36]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix + '-Eval-Step'
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
    ],
   steps=[step_eval, step_cond],
)

### 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [37]:
import json


definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [38]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase02-eval-step',
 'ResponseMetadata': {'RequestId': '33430c65-5fb7-4fd8-b23e-0855341c9a2f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '33430c65-5fb7-4fd8-b23e-0855341c9a2f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '122',
   'date': 'Wed, 25 Aug 2021 13:00:06 GMT'},
  'RetryAttempts': 0}}

In [39]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [40]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase02-eval-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase02-eval-step/execution/dx3sqjkzgnib',
 'PipelineExecutionDisplayName': 'execution-1629896407599',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 8, 25, 13, 0, 7, 536000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 25, 13, 0, 7, 536000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'fc693302-41db-4202-8ba7-f1d604afc7a7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fc693302-41db-4202-8ba7-f1d604afc7a7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '473',
   'date': 'Wed, 25 Aug 2021 13:00:07 GMT'},
  'RetryAttempts': 0}}

In [41]:
execution.wait()

In [42]:
execution.list_steps()

[{'StepName': 'FraudMeticCond',
  'StartTime': datetime.datetime(2021, 8, 25, 13, 4, 23, 152000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 25, 13, 4, 23, 453000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Condition': {'Outcome': 'True'}}},
 {'StepName': 'FraudEval',
  'StartTime': datetime.datetime(2021, 8, 25, 13, 0, 8, 546000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 8, 25, 13, 4, 22, 753000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:processing-job/pipelines-dx3sqjkzgnib-fraudeval-juscffzygm'}}}]

# 4. 리소스 정리: 파이프라인
- 위에서 생성한 파이프라인을 제거 합니다.
- isDeletePipeline=False, verbose=Fasle
    - 파이프라인을 지우지 않고, 존재하는지 확인 합니다.
- isDeletePipeline=False, verbose=True
    - 파이프라인의 정의를 자세하 확인 합니다.
- isDeletePipeline=True, verbose=True or False
    - 파이프라인을 삭제 합니다.

In [43]:
from src.p_utils import clean_pipeline

# clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=True, verbose=False)   

pipeline sagemaker-pipeline-step-by-step-phase02-Eval-Step exists
pipeline sagemaker-pipeline-step-by-step-phase02-Eval-Step is deleted
